In [1]:
%reload_ext watermark
%watermark -v -p tensorflow,pandas,numpy,cv2,sklearn

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.34.0

tensorflow: 2.8.0
pandas    : 1.4.1
numpy     : 1.22.3
cv2       : 4.6.0
sklearn   : 1.0.2



In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2,datetime
print(datetime.datetime.today())

2022-12-16 04:43:57.893016


In [3]:
def my_generator(df):
    def img_load(file):
        img = cv2.imread(file,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320, 240))
        return img
    
    def generator():
        for label, path in df.values:
            img = img_load(path)
            y=np.zeros((3))
            if label =="Open":
                y[0]=1
            elif label=="Closing":
                y[1]=1
            else: 
                y[2]=1
            yield img, y
    return generator
print(datetime.datetime.today())

2022-12-16 04:43:57.927243


In [4]:
train=pd.read_csv("눈깜빡임분류_train.tsv",sep='\t')
valid=pd.read_csv("눈깜빡임분류_valid.tsv",sep='\t')
print(datetime.datetime.today())

2022-12-16 04:44:00.955697


In [5]:
tr_dataset=tf.data.Dataset.from_generator(
        my_generator(train),
        output_shapes=((240,320,3), 3),
        output_types=("float32", "float32"))
tr_gen =tr_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset=tf.data.Dataset.from_generator(
        my_generator(valid),
        output_shapes=((240,320,3), 3),
        output_types=("float32", "float32"))
val_gen =val_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)
print(datetime.datetime.today())

2022-12-16 04:44:03.560762


2022-12-16 04:44:02.332573: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 04:44:02.854923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46723 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6


## 모델 학습

In [6]:
def my_cnn_model():
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    cnn_model = tf.keras.Sequential()
    cnn_model.add(tf.keras.layers.Input((240,320,3)))
    cnn_model.add(tf.keras.layers.Rescaling(1./255))
    cnn_model.add(tf.keras.layers.Resizing(256,320,name='resize'))

    cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Flatten())
    cnn_model.add(tf.keras.layers.Dense(256, activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.Dropout(0.2))
    cnn_model.add(tf.keras.layers.Dense(3, activation='softmax'))
    return cnn_model
print(datetime.datetime.today())

2022-12-16 04:44:03.573385


In [7]:
print(datetime.datetime.today())
tf.keras.backend.clear_session()
tf.random.set_seed(42)
model=my_cnn_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=.01),
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)
model.fit(tr_gen, epochs=3, validation_data=val_gen,
      verbose=1)
print(datetime.datetime.today())

2022-12-16 04:44:03.580279
Epoch 1/3


2022-12-16 04:44:05.208764: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-12-16 04:44:06.036536: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-12-16 04:44:08.985694: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2737/2737 [==============================] - 21816s 8s/step - loss: 0.8149 - accuracy: 0.6217 - val_loss: 0.9321 - val_accuracy: 0.5300
Epoch 2/3
2737/2737 [==============================] - 19438s 7s/step - loss: 0.7452 - accuracy: 0.6509 - val_loss: 0.8160 - val_accuracy: 0.6552
Epoch 3/3
2737/2737 [==============================] - 18872s 7s/step - loss: 0.7203 - accuracy: 0.6665 - val_loss: 0.7211 - val_accuracy: 0.7035
2022-12-16 21:26:11.870828


In [8]:
model.save("눈깜빡임분류_v0_1.h5")

## 모델 검증

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2,datetime
from sklearn.metrics import roc_auc_score
print(datetime.datetime.today())

2022-12-18 09:42:56.325971


In [2]:
print(datetime.datetime.today())
model=tf.keras.models.load_model("눈깜빡임분류_v0_1.h5")
print(datetime.datetime.today())

2022-12-18 09:42:56.331549


2022-12-18 09:42:57.798017: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 09:42:58.305826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46723 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6


2022-12-18 09:43:00.885840


In [3]:
def my_generator(df):
    def img_load(file):
        img = cv2.imread(file,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (320, 240))
        return img
    
    def generator():
        for label, path in df.values:
            img = img_load(path)
            y=np.zeros((3))
            if label =="Open":
                y[0]=1
            elif label=="Closing":
                y[1]=1
            else: 
                y[2]=1
            yield img, y
    return generator

In [4]:
test =pd.read_csv("눈깜빡임분류_test.tsv",sep='\t')
te_dataset=tf.data.Dataset.from_generator(
        my_generator(test),
        output_shapes=((240,320,3), 3),
        output_types=("float32", "float32"))
te_gen =te_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)
print(datetime.datetime.today())

2022-12-18 09:43:01.441533


In [5]:
predict_values=model.predict(te_gen,verbose=1)
print(datetime.datetime.today())

2022-12-18 09:43:02.581524: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-12-18 09:43:03.396976: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


      1/Unknown - 5s 5s/step

2022-12-18 09:43:06.187546: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


551/551 [==============================] - 5050s 9s/step
2022-12-18 11:07:11.969785


In [6]:
test['patient_id']=['/'.join(i.split('/')[5:7]) 
                    for i in test.file_path]
test['predict_values']=np.array(['Open','Closing','Closed'])[
    np.argmax(predict_values,axis=1)]
test[['patient_id','blink_tagging','predict_values']].to_csv(
    "눈깜빡임분류_결과값.tsv",index=False,sep='\t')

In [7]:
y_true=np.zeros((test.shape[0],3))
for i,label in enumerate(test.blink_tagging):
    if label=="Open":
        y_true[i,0]=1
    elif label=="Closing":
        y_true[i,1]=1
    else:
        y_true[i,2]=1

In [11]:
test[['Open_true','Closing_true','Closed_true']]=y_true
test[['Open_prob','Closing_prob','Closed_prob']]=predict_values

In [14]:
test.loc[:,['patient_id','Open_true','Closing_true','Closed_true',
      'Open_prob','Closing_prob','Closed_prob']].to_csv(
    '눈깜빡임분류_성능평가계산사용값.tsv',
    index=False,sep='\t')

In [15]:
score={"Open":roc_auc_score(y_true[:,0],predict_values[:,0]),
 "Closing":roc_auc_score(y_true[:,1],predict_values[:,1]),
 "Closed":roc_auc_score(y_true[:,2],predict_values[:,2])}
print(score,"total : ",np.mean(list(score.values())))
print(datetime.datetime.today())

{'Open': 0.8777455296975807, 'Closing': 0.7636314496000453, 'Closed': 0.844819209665828} total :  0.828732062987818
2022-12-18 11:11:02.352505


In [17]:
pd.DataFrame({"AUC":[np.mean(list(score.values()))]}).to_csv(
    "눈깜빡임분류_AUC.tsv",index=False,sep='\t')
print(datetime.datetime.today())

2022-12-18 11:11:37.018345
